# Demonstrate intake access to UM DYAMOND3 simulations from zarr on JASMIN object store: v5 version

### 7/5/25

* **Fourth look at simulations for WCRP Hackathon UK Node. v5 is the final version of the global data**
* **Zoom level 10 (regional) or 9 (n1280) may not be complete**
* **Stores are not complete**
* **However these stores will be filled in as the simulation data becomes avialable. We will let other nodes know when they are complete**
* Shows the hierarchy of simulations that will/should be available at the UK node, from global to regional.
* You can see the URLs which are active in the `cat` catalog.
* Contact mark.muetzelfeldt@reading.ac.uk for more info.

## Simulations

* glm: global model. n1280 is approx. 10 km res (stored at zoom 9), n2560 is 5 km (zoom 10). Regional simulations are at 4.4 km (zoom 10).
* Regional: Africa, South East Asia, South America, Cyclic Tropical Channel
* Settings:
    * CoMA9: CoMorph global,
    * RAL3: Regional Atmosphere Land 3
    * GAL9: Global Atmosphere Land 9
    * RAL3p3: RAL3.3
    * CoMA9_TBv1: CoMA9 TrailBlazer v1

## Technical

* All data stored as healpix, including regional.
* Regional simulations only store active chunks.
* Regional data necessarily has `nan`s to represent data outside the domain. This can cause issues when calculating domain means at different zooms. The `weights` field should help mitigate this (instructions to follow).
* There are two stores for each zoom level, one for `PT1H` (2D) and `PT3H` (3D) variables. All simulations are in the `sims` variable.
* Calling `ds = ds.compute()` downloads the data from JASMIN. This can be slow and/or fail with a server error. Try again if this happens.
* Tested using this Python conda env: https://github.com/digital-earths-global-hackathon/tools/blob/main/python_envs/environment.yaml (with some extra packages).
    * You can install with:
    * `wget https://raw.githubusercontent.com/digital-earths-global-hackathon/tools/refs/heads/main/python_envs/environment.yaml`
    * <edit last line of environment.yaml to be the name of your new env, e.g. hackathon_env>
    * `conda env create -f environment.yaml`
* Not all variables in the standard protocol are present - I have included those that are.
* I believe there is a plotting issue at lon=0 - and that data is OK.

## Issues

* CTC simulations where I think there is a genuine regridding issue at lon=0 where healpix data may be missing (simulations are fine)
* Data not complete at zooms 9-0 (regional) or 8-0 (n1280), although empty zarr stores are present
* No data for most times for regional

In [ ]:
import math as maths

import cartopy.crs as ccrs
import intake
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import easygems.healpix as egh

In [ ]:
import warnings

# Suppress specific FutureWarnings matching the message pattern when using cat[...].to_dask()
warnings.filterwarnings(
    "ignore",
    message=".*The return type of `Dataset.dims` will be changed.*",
    category=FutureWarning,
)

In [ ]:
cat = intake.open_catalog('https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml')['online']

In [ ]:
# Show UM sims:
[key for key in cat if key.startswith('um_')]

## Demonstrate access using 5km RAL3 (um_glm_n2560_RAL3p3)

* This is the main hackathon simulation, and is complete as of 6/5/25 (i.e. all times/zooms stored)

In [ ]:
sim = cat['um_glm_n2560_RAL3p3']

In [ ]:
# Show example URL
sim(zoom=8).urlpath

In [ ]:
# Open a dataset.
ds = sim(zoom=8, time='PT1H').to_dask()

In [ ]:
# Explore dataset. No data downloaded at this point, only metadata.
ds

In [ ]:
# Quick plot of global T at 1.5m
egh.healpix_show(ds.isel(time=0).tas)

In [ ]:
def plot_all_fields(ds_plot):
    """Plot all fields for a given dataset. Assumes that each field is 2D - i.e. sel(time=..., [pressure=...]) has been applied"""
    zoom = int(np.log2(ds_plot.crs.attrs['healpix_nside']))
    projection = ccrs.Robinson(central_longitude=0)
    rows = maths.ceil(len(ds_plot.data_vars) / 4)
    fig, axes = plt.subplots(rows, 4, figsize=(30, rows * 20 / 6), subplot_kw={'projection': projection}, layout='constrained')
    if 'pressure' in ds_plot.coords:
        plt.suptitle(f'{ds.simulation} z{zoom} @{float(ds_plot.pressure)}hPa')
    else:
        plt.suptitle(f'{ds.simulation} z{zoom}')
            
    for ax, (name, da) in zip(axes.flatten(), ds_plot.data_vars.items()):
        time = pd.Timestamp(ds.time.values[0])
    
        if abs(da.max() + da.min()) / (da.max() - da.min()) < 0.5:
            # data looks like it needs a diverging cmap.
            # figure out some nice bounds.
            pl, pu = np.percentile(da.values[~np.isnan(da.values)], [2, 98])
            vmax = np.abs([pl, pu]).max()
            kwargs = dict(
                cmap='bwr',
                vmin=-vmax,
                vmax=vmax,
            )
        else:
            kwargs = {}
        ax.set_title(f'time: {time} - {name}')
        ax.set_global()
        im = egh.healpix_show(da, ax=ax, **kwargs);
        long_name = da.long_name
            
        plt.colorbar(im, label=f'{long_name} ({da.attrs.get("units", "-")})')
        ax.coastlines()

In [ ]:
# Download the requested data for plotting.
ds3d = sim(time='PT3H', zoom=8).to_dask().sel(time='2020-01-20 03:00', pressure=500).compute()

In [ ]:
plot_all_fields(ds3d)

In [ ]:
ds2d = sim(time='PT1H', zoom=8).to_dask().sel(time='2020-01-20 03:00').compute()

In [ ]:
plot_all_fields(ds2d)

## Explore the Africa regional sim

* Not all data available, use zoom 10 (highest resolution)

In [ ]:
ds_africa = cat['um_Africa_km4p4_RAL3P3_n1280_GAL9_nest'](time='PT1H', zoom=10).to_dask().sel(time='2020-01-20 03:00').compute()

In [ ]:
plot_all_fields(ds_africa)

## Plot all sims in single plot

In [ ]:
um_sims = [k for k in cat.keys() if k.startswith('um_')]

In [ ]:
# Get dataset for all available sims.
dss = {}
for simname in um_sims:
    zoom = 9 if 'um_glm_n1280' in simname else 10
    try:
        dss[simname] = cat[simname](time='PT1H', zoom=zoom).to_dask()
    except Exception as e:
        print(f'Could not load {simname}')
        print(e)

In [ ]:
# Sort to nicer order for plotting.
def sorter(simname):
    if 'um_glm_n2560' in simname:
        return 'A'
    elif 'um_glm_n1280' in simname:
        return 'AA'
    else:
        return simname

dss = {s: dss[s] for s in sorted(dss.keys(), key=sorter)}

In [ ]:
def plot_var(plot_dss, var, time, **plot_kwargs):
    """Plot given var from each dataset."""
    rows = maths.ceil(len(plot_dss) / 3)
    projection = ccrs.Robinson(central_longitude=0)
    fig, axes = plt.subplots(rows, 3, figsize=(30, 5 * rows), subplot_kw={'projection': projection}, layout='constrained')
            
    for ax, (name, ds) in zip(axes.flatten(), plot_dss.items()):
        time = pd.Timestamp(ds.time.values[0])
        da = ds[var].sel(time=time).compute()
    
        if abs(da.max() + da.min()) / (da.max() - da.min()) < 0.5:
            # data looks like it needs a diverging cmap.
            # figure out some nice bounds.
            pl, pu = np.percentile(da.values[~np.isnan(da.values)], [2, 98])
            vmax = np.abs([pl, pu]).max()
            kwargs = dict(
                cmap='bwr',
                vmin=-vmax,
                vmax=vmax,
            )
        else:
            kwargs = {}
        kwargs.update(plot_kwargs)
        ax.set_title(f'time: {time} - {name}')
        ax.set_global()
        if ds.attrs.get('regional', False):
            # Display the active chunks for any regional data.
            ds_ones = xr.Dataset({'ones': (['cell'], np.ones_like(ds.isel(time=0).tas))}, coords={'cell': ds.cell}).assign_coords(crs=ds.crs)
            egh.healpix_show(ds_ones.ones, ax=ax)
        im = egh.healpix_show(da, ax=ax, **kwargs);
        long_name = da.long_name
            
        plt.colorbar(im, label=f'{long_name} ({da.attrs.get("units", "-")})')
        ax.coastlines()

In [ ]:
# Display tas/air_temperature for all available sims.
# For regional data, this also shows the active chunks (purply jagged outline). Only active chunks are saved to minimize memory reqs on host computer when loading data.
plot_var(dss, 'tas', '2020-01-20 10:00', vmin=215, vmax=310)